In [0]:
%run "/Workspace/ETL_Arquitetura_Medalhão/00.Configuração/configuração"

Catalogo: workspace
Schemas: bronze silver gold


In [0]:
from pyspark.sql import functions as F, Window

In [0]:
df = spark.table(f"{CATALOG}.{SILVER}.economia_consolidada")

In [0]:
w = Window.orderBy("data")

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w_acumulada = (
    Window
    .orderBy("data")
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)
)


In [0]:
gold = (
    df
    .withColumn("ipca_anterior", F.lag("ipca").over(w))
    .withColumn("boi_anterior", F.lag("boi_gordo").over(w))
    .withColumn("fator_ipca", ((F.col("IPCA")/100)+1))
    .withColumn("fator_ipca_acumulado",F.exp(F.sum(F.log(F.col("fator_ipca"))).over(w_acumulada)))
    .withColumn("ipca_acumulado",(F.col("fator_ipca_acumulado")-1)*100)
    .withColumn("variacao_boi", (F.col("boi_gordo") - F.col("boi_anterior")) / F.col("boi_anterior") * 100)
    .drop("ipca_anterior", "boi_anterior","fator_ipca","fator_ipca_acumulado", "variacao_ipca")
)

display(gold)



/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


data,IPCA,boi_gordo,data_ingestao,ipca_acumulado,variacao_boi
2025-01-01,0.16,324.95,2025-12-27T18:24:24.338Z,0.16000000000000458,null
2025-02-01,1.31,319.21,2025-12-27T18:24:24.338Z,1.4720960000000227,-1.7664256039390707
2025-03-01,0.56,312.47,2025-12-27T18:24:24.338Z,2.0403397376000276,-2.1114626734751267
2025-04-01,0.43,323.96,2025-12-27T18:24:24.338Z,2.479113198471694,3.677153006688626
2025-05-01,0.26,308.15,2025-12-27T18:24:24.338Z,2.745558892787714,-4.88023212742314
2025-06-01,0.24,313.51,2025-12-27T18:24:24.338Z,2.9921482341304095,1.7394126237222178
2025-07-01,0.26,299.97,2025-12-27T18:24:24.338Z,3.2599278195391435,-4.318841504258226
2025-08-01,-0.11,307.25,2025-12-27T18:24:24.338Z,3.1463418989376546,2.4269093576024177
2025-09-01,0.48,307.87,2025-12-27T18:24:24.338Z,3.6414443400525354,0.20179007323026998
2025-10-01,0.09,310.51,2025-12-27T18:24:24.338Z,3.7347216399585825,0.8575047909832028


In [0]:
gold.write.format("delta").mode("overwrite").saveAsTable(f"{CATALOG}.{GOLD}.insights")

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
%sql
ALTER TABLE workspace.gold.insights
SET TBLPROPERTIES (
  'delta.columnMapping.mode' = 'name'
);


In [0]:
%sql

SELECT * FROM workspace.gold.insights

data,IPCA,boi_gordo,data_ingestao,variacao_boi,ipca_acumulado
2025-01-01,0.16,324.95,2025-12-27T18:24:24.338Z,null,0.16000000000000458
2025-02-01,1.31,319.21,2025-12-27T18:24:24.338Z,-1.7664256039390707,1.4720960000000227
2025-03-01,0.56,312.47,2025-12-27T18:24:24.338Z,-2.1114626734751267,2.0403397376000276
2025-04-01,0.43,323.96,2025-12-27T18:24:24.338Z,3.677153006688626,2.479113198471694
2025-05-01,0.26,308.15,2025-12-27T18:24:24.338Z,-4.88023212742314,2.745558892787714
2025-06-01,0.24,313.51,2025-12-27T18:24:24.338Z,1.7394126237222178,2.9921482341304095
2025-07-01,0.26,299.97,2025-12-27T18:24:24.338Z,-4.318841504258226,3.2599278195391435
2025-08-01,-0.11,307.25,2025-12-27T18:24:24.338Z,2.4269093576024177,3.1463418989376546
2025-09-01,0.48,307.87,2025-12-27T18:24:24.338Z,0.20179007323026998,3.6414443400525354
2025-10-01,0.09,310.51,2025-12-27T18:24:24.338Z,0.8575047909832028,3.7347216399585825


In [0]:
%sql
CREATE OR REPLACE VIEW workspace.gold.vw_insights AS
SELECT * FROM workspace.gold.insights;

In [0]:
%sql
SELECT data, variacao_boi, ipca_acumulado FROM workspace.gold.vw_insights

data,variacao_boi,ipca_acumulado
2025-01-01,null,0.16000000000000458
2025-02-01,-1.7664256039390707,1.4720960000000227
2025-03-01,-2.1114626734751267,2.0403397376000276
2025-04-01,3.677153006688626,2.479113198471694
2025-05-01,-4.88023212742314,2.745558892787714
2025-06-01,1.7394126237222178,2.9921482341304095
2025-07-01,-4.318841504258226,3.2599278195391435
2025-08-01,2.4269093576024177,3.1463418989376546
2025-09-01,0.20179007323026998,3.6414443400525354
2025-10-01,0.8575047909832028,3.7347216399585825


In [0]:
%sql
SELECT data, ipca, variacao_boi, CASE 
WHEN variacao_boi > ipca THEN 'Boi acima da inflação'
WHEN variacao_boi < ipca THEN 'Boi abaixo da inflação'
ELSE 'Empate'
END AS status_relativo

FROM workspace.gold.vw_insights;

data,ipca,variacao_boi,status_relativo
2025-01-01,0.16,null,Empate
2025-02-01,1.31,-1.7664256039390707,Boi abaixo da inflação
2025-03-01,0.56,-2.1114626734751267,Boi abaixo da inflação
2025-04-01,0.43,3.677153006688626,Boi acima da inflação
2025-05-01,0.26,-4.88023212742314,Boi abaixo da inflação
2025-06-01,0.24,1.7394126237222178,Boi acima da inflação
2025-07-01,0.26,-4.318841504258226,Boi abaixo da inflação
2025-08-01,-0.11,2.4269093576024177,Boi acima da inflação
2025-09-01,0.48,0.20179007323026998,Boi abaixo da inflação
2025-10-01,0.09,0.8575047909832028,Boi acima da inflação


In [0]:
%sql
SELECT data, boi_gordo AS boi_nominal,
ipca_acumulado,
boi_gordo/(1+ipca_acumulado/100) AS boi_preco_real
FROM workspace.gold.vw_insights

data,boi_nominal,ipca_acumulado,boi_preco_real
2025-01-01,324.95,0.16000000000000458,324.43091054313095
2025-02-01,319.21,1.4720960000000227,314.57909374415595
2025-03-01,312.47,2.0403397376000276,306.22203023189314
2025-04-01,323.96,2.479113198471694,316.12295412098797
2025-05-01,308.15,2.745558892787714,299.9156394891446
2025-06-01,313.51,2.9921482341304095,304.40184555360736
2025-07-01,299.97,3.2599278195391435,290.4999125355178
2025-08-01,307.25,3.1463418989376546,297.87774761905007
2025-09-01,307.87,3.6414443400525354,297.0529810351386
2025-10-01,310.51,3.7347216399585825,299.33082683512174


In [0]:
%sql
SELECT
  STDDEV(variacao_boi) AS volatilidade_boi,
  STDDEV(ipca) AS volatilidade_ipca
FROM workspace.gold.vw_insights;

volatilidade_boi,volatilidade_ipca
3.1113498041988823,0.36898386266758454


In [0]:
%sql
SELECT
    data,
    variacao_boi,
    LAG(variacao_boi,1) OVER (ORDER BY data) AS variacao_boi_lag1

FROM workspace.gold.vw_insights

data,variacao_boi,variacao_boi_lag1
2025-01-01,null,null
2025-02-01,-1.7664256039390707,null
2025-03-01,-2.1114626734751267,-1.7664256039390707
2025-04-01,3.677153006688626,-2.1114626734751267
2025-05-01,-4.88023212742314,3.677153006688626
2025-06-01,1.7394126237222178,-4.88023212742314
2025-07-01,-4.318841504258226,1.7394126237222178
2025-08-01,2.4269093576024177,-4.318841504258226
2025-09-01,0.20179007323026998,2.4269093576024177
2025-10-01,0.8575047909832028,0.20179007323026998


In [0]:
%sql
SELECT
    CORR(variacao_boi, ipca) AS correlacao_boi_ipca,
    CORR(boi_gordo, ipca_acumulado) AS correlacao_boi_ipca_acumulado
FROM workspace.gold.vw_insights

correlacao_boi_ipca,correlacao_boi_ipca_acumulado
-0.2932050278484972,-0.5174755556451481


In [0]:
%sql
SELECT
  data,
  variacao_boi,
  ipca
FROM workspace.gold.vw_insights
WHERE ABS(variacao_boi) > 3;

data,variacao_boi,ipca
2025-04-01,3.677153006688626,0.43
2025-05-01,-4.88023212742314,0.26
2025-07-01,-4.318841504258226,0.26
2025-11-01,3.7261279829957146,0.18
